<a href="https://colab.research.google.com/github/caroline-dainezi-fatec/fatec_PLN_Codes/blob/main/Aula8_IntroducaoMLParaPLN/%5BPLN%5D_Aula_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 8 - Introdução a Machine Learning para PLN

## Exemplo 01 - Aplicação do modelo de Naives em um texto

### Explicação do Código
O código implementa um classificador probabilístico simples para prever sentimentos (positivo ou negativo) em frases curtas.

1. Na criação do Corpus, é feita uma lista de frases categorizadas em positivo ou negativo.

2. A função `preprocess_text` converte textos para minúsculas, remove caracteres não alfabéticos e retorna uma lista de palavras pré-processadas.

3. Passo 3 - Calculando Probabilidades

  O passo 3 realiza a contagem de frequências e o cálculo das probabilidades a priori e condicionais (etapas essenciais para classificar textos com base no modelo probabilístico).

  - Variáveis iniciais:
    - `class_counts`: Um `Counter` que armazena o número de exemplos em cada classe (positivo ou negativo).
    - `word_counts`: Registra as frequências das palavras em cada classe.
    - `total_words`: Mantém a contagem total de palavras em cada classe.

  - Para cada frase do corpus (`preprocessed_corpus`), o loop `for`:
      - Incrementa o contador de exemplos para a classe correspondente em `class_counts`.
      - Depois, para cada palavra da frase:
        - Incrementa a contagem da palavra na classe em `word_counts`.
        - Atualiza o total de palavras da classe em `total_words`.

  - A variável `prior_probabilities` calcula a probabilidade de ocorrência de cada classe com base na proporção de exemplos daquela classe no corpus.

  - A função `conditional_probability` retorna a probabilidade de uma palavra estar associada a uma classe específica, considerando suavização de Laplace para evitar probabilidades zero. Seus parâmetros são:
    - `word`: A palavra a ser avaliada.
    - `label`: A classe associada (*positivo* ou *negativo*).
    - `alpha`: O fator de suavização (valor padrão é 1).

4. Passo 4 - Classificar um Novo Texto

  O passo 4 implementa o processo de classificação de um novo texto utilizando as probabilidades a priori e condicionais calculadas no passo 3.

  - Pré-processamento do texto:
    - A função `predict` recebe um texto como entrada.
    - O texto é pré-processado com a função `preprocess_text` do passo 2 para tokenização e normalização.

  - Um dicionário `probabilities` é inicializado para armazenar a probabilidade de o texto pertencer a cada classe.
    - Para cada classe existente em `class_counts`, a probabilidade inicial da classe é definida como sua probabilidade a priori (`prior_probabilities` do passo 3).
    - Para cada palavra no texto, multiplica a probabilidade da classe pela probabilidade condicional da palavra, calculada com a função `conditional_probability`.

  - A classe com a maior probabilidade no dicionário `probabilities` é selecionada como a predição final.
    - A função retorna a classe predita e as probabilidades calculadas para todas as classes.

5. Com o texto "*Eu amo resolver bugs*", o classificador prevê o rótulo positivo e exibe as probabilidades calculadas.

In [2]:
# Passo 1 - Criar o Corpus

corpus = [
    ("Eu amo PLN", "positivo"),
    ("Eu odeio bugs", "negativo"),
    ("Amo resolver problemas", "positivo"),
    ("Odeio erros", "negativo")
]

# Passo 2 - Processar o Texto
import re
from collections import defaultdict, Counter

def preprocess_text(text):
  return re.findall(r'\b\w+\b', text.lower())

preprocessed_corpus = [(preprocess_text(text), label) for text, label in corpus]
print(preprocessed_corpus)

# Passo 3 - Calculando probabilidades
  # este passo precisa estar explicado
class_counts = Counter() # Contagem das classes
word_counts = defaultdict(Counter)
total_words = defaultdict(int)

for words, label in preprocessed_corpus:
  class_counts[label] += 1
  for word in words:
    word_counts[label][word] += 1
    total_words[label]

total_examples = sum(class_counts.values())
prior_probabilities = {cls: count / total_examples for cls, count in class_counts.items()}
def conditional_probability(word, label, alpha=1):
  return (word_counts[label][word] + alpha) / (total_words[label] + alpha * len(word_counts[label]))

# Passo 4 - Classificar um novo Texto
  # este passo precisa estar explicado
def predict(text):
  words = preprocess_text(text)
  probabilities = {}

  for label in class_counts.keys():
    probabilities[label] = prior_probabilities[label]
    for word in words:
      probabilities[label] *= conditional_probability(word, label)
  return max(probabilities, key=probabilities.get), probabilities

# Passo 5 - Teste com um novo texto
novo_texto = "Eu amo resolver bugs"
classe, probs = predict(novo_texto)

print(f'Texto: "{novo_texto}"')
print(f'Classe prevista: {classe}')
print(f'Probabilidades:')
for label, prob in probs.items():
  print(f" {label}: {prob}")

[(['eu', 'amo', 'pln'], 'positivo'), (['eu', 'odeio', 'bugs'], 'negativo'), (['amo', 'resolver', 'problemas'], 'positivo'), (['odeio', 'erros'], 'negativo')]
Texto: "Eu amo resolver bugs"
Classe prevista: positivo
Probabilidades:
 positivo: 0.009600000000000001
 negativo: 0.0078125


## Exemplo 2 - Modelo de SVM

### Explicação do código

1. Primeiro, são importadas as bibliotecas necessárias para a implementação do modelo de classificação de texto utilizando o método SVC: `TfidfVectorizer`, `SVC`, `train_test_split` e `classification_report`.

2. O código define um conjunto de dados de exemplo (corpus) e suas respectivas classes, que indicam se a frase é "positivo" ou "negativo".

3. Transforma os textos em vetores numéricos para que possam ser usados pelo modelo de machine learning:

  - O `TfidfVectorizer` converte cada documento em um vetor baseado na técnica TF-IDF, que considera a importância das palavras dentro do corpus.
  - `X`: O conjunto de dados transformados em vetores, pronto para ser usado no treinamento.
  - `y`: A variável com as classes correspondentes a cada frase.

4. Os dados são divididos entre treinamento e teste e o modelo é treinado:

  - `train_test_split` divide os dados `X` e `y` em dois conjuntos: 70% para treinamento e 30% para teste (`test_size=0.3`).
  - O modelo SVM (`SVC(kernel='linear)`) é treinado com os dados de treinamento `X_train` e `y_train`.

5. Avalia o modelo no conjunto de teste `X_test`, e o `classification_report` é utilizado para gerar um relatório que mostra a performance do modelo.

In [3]:
# Passo 1 - Importação das bibliotecas a serem utilizadas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Passo 2 - Dados de Exemplo
corpus = [
    "Eu amo PLN", "Eu odeio bugs", "Eu amo resolver problemas",
    "Odeio erros", "Amo programação", "Não gosto de falhas"
]
classes = ["negativo", "negativo", "positivo", "negativo", "positivo", "negativo"]

# Passo 3 - Pré-processamento e vetorização
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
y = classes

# Passo 4 - Dividir os dados e terinar o modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Passo 5 - Avaliar o modelo
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negativo       1.00      0.50      0.67         2
    positivo       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.50      0.25      0.33         2
weighted avg       1.00      0.50      0.67         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Exemplo 3 - Comparação dos dois modelos

### Explicação do código

1. São importadas as bibliotecas necessárias para o pré-processamento de texto e para a construção do modelo de classificação. Isso inclui a `TfidfVectorizer`, a `train_test_split`, a `MultinomialNB` (Naive Bayes) e `SVC` (SVM) para construir os modelos, e `classification_report`.

2. Preparação dos Dados:
  - O dataset de `movie_reviews` da biblioteca `NLTK` é carregado (resenhas de filmes rotuladas como "positivo" ou "negativo").
  - Os documentos são coletados, onde cada texto é combinado com seu respectivo rótulo.
  - Os textos e as classes são separados e as classes são transformadas para valores numéricos (0 para "negativo" e 1 para "positivo") usando o `LabelEncoder`.

3. O `TfidfVectorizer` é configurado para limitar a 5.000 das palavras mais frequentes do corpus. Ele também é ajustado e aplicado aos textos de treinamento (`X_train`) e, em seguida, os textos de teste (`X_test`) são transformados.

4. Ambos os modelos `MultinomialNB` (Naive Bayes) e `SVC` (SVM) são treinado com os dados de treino e fazem previsões sobre os dados de teste.

5. Para cada modelo, as previsões são avaliadas usando `classification_report`, que gera métricas como precisão, recall e f1-score, permitindo comparar a performance de ambos os modelos no conjunto de teste.

In [1]:
# 1. Importar Bibliotecas
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report

  # Baixar o dataset de exemplo
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

# 2. Preparação dos dados

  # Coleta de textos e classes
documents = [
    (" ".join(movie_reviews.words(fileid)), category)
    for category in movie_reviews.categories()
    for fileid in movie_reviews.fileids(category)
]

  # Separar textos e rótulos
texts, labels = zip(*documents)

  # Transformar rótulos (positivo/negativo) em 0 e 1
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

  # Dividir dados em treino e teste
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.3, random_state=42)

# 3. Representação do texto com TF-IDF
  # Criar o vetorizador TF_IDF
vectorizer = TfidfVectorizer(max_features=5000) # Limitar a 5.000 palavras mais comuns

  # Ajustar e transformar os textos
X_train = vectorizer.fit_transform(texts_train)
X_test = vectorizer.transform(texts_test)

# 4. Treinar os modelos

  # Treinamento do Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, labels_train)

  # Predição
nb_predictions = nb_model.predict(X_test)

  # Treinamento do SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, labels_train)

  # Predição
svm_predictions = svm_model.predict(X_test)

# 5. Avaliação

  # Avaliação do Naive Bayes
print("Naive Bayes Performance:")
print(classification_report(labels_test, nb_predictions, target_names=label_encoder.classes_))

  # Avaliação do SVM
print("SVM Performance:")
print(classification_report(labels_test, svm_predictions, target_names=label_encoder.classes_))

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


Naive Bayes Performance:
              precision    recall  f1-score   support

         neg       0.79      0.84      0.81       302
         pos       0.82      0.77      0.80       298

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600

SVM Performance:
              precision    recall  f1-score   support

         neg       0.82      0.80      0.81       302
         pos       0.81      0.82      0.81       298

    accuracy                           0.81       600
   macro avg       0.81      0.81      0.81       600
weighted avg       0.81      0.81      0.81       600

